<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Major Neural Network Architectures Challenge
## *Data Science Unit 4 Sprint 3 Challenge*

In this sprint challenge, you'll explore some of the cutting edge of Data Science. This week we studied several famous neural network architectures: 
recurrent neural networks (RNNs), long short-term memory (LSTMs), convolutional neural networks (CNNs), and Autoencoders. In this sprint challenge, you will revisit these models. Remember, we are testing your knowledge of these architectures not your ability to fit a model with high accuracy. 

__*Caution:*__  these approaches can be pretty heavy computationally. All problems were designed so that you should be able to achieve results within at most 5-10 minutes of runtime on SageMaker, Colab or a comparable environment. If something is running longer, doublecheck your approach!

## Challenge Objectives
*You should be able to:*
* <a href="#p1">Part 1</a>: Train a LSTM classification model
* <a href="#p2">Part 2</a>: Utilize a pre-trained CNN for objective detection
* <a href="#p3">Part 3</a>: Describe the components of an autoencoder
* <a href="#p4">Part 4</a>: Describe yourself as a Data Science and elucidate your vision of AI

<a id="p1"></a>
## Part 1 - RNNs

Use an RNN/LSTM to fit a multi-class classification model on reuters news articles to distinguish topics of articles. The data is already encoded properly for use in an RNN model. 

Your Tasks: 
- Use Keras to fit a predictive model, classifying news articles into topics. 
- Report your overall score and accuracy

For reference, the [Keras IMDB sentiment classification example](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py) will be useful, as well the RNN code we used in class.

__*Note:*__  Focus on getting a running model, not on maxing accuracy with extreme data size or epoch numbers. Only revisit and push accuracy if you get everything else done!

In [13]:
import warnings
warnings.filterwarnings('ignore', category = FutureWarning)

import numpy as np

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

from keras.datasets import reuters

In [90]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(path="reuters.npz",
                                                         num_words=None,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=113,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)

In [91]:
# Demo of encoding

word_index = reuters.get_word_index(path="reuters_word_index.json")

print(f"Iran is encoded as {word_index['iran']} in the data")
print(f"London is encoded as {word_index['london']} in the data")
print("Words are encoded as numbers in our dataset.")

Iran is encoded as 779 in the data
London is encoded as 544 in the data
Words are encoded as numbers in our dataset.


In [92]:
len(word_index)

30979

In [96]:
batch_size = 46
max_features = len(word_index.values())+1
maxlen = 200

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

8982 train sequences
2246 test sequences
Pad sequences (samples x time)
X_train shape: (8982, 200)
X_test shape: (2246, 200)


In [97]:
#model architechture
model = Sequential()

model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.25, recurrent_dropout=0.2))
model.add(Dense(len(word_index), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         3965440   
_________________________________________________________________
lstm_17 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_17 (Dense)             (None, 30979)             3996291   
Total params: 8,093,315
Trainable params: 8,093,315
Non-trainable params: 0
_________________________________________________________________


In [98]:
# You should only run this cell once your model has been properly configured
print('Train...')
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=2,
          validation_data=(X_test, y_test))

score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 8982 samples, validate on 2246 samples
Epoch 1/2
8982/8982 [==============================] - 118s 13ms/sample - loss: 3.5457 - acc: 0.3484 - val_loss: 2.4429 - val_acc: 0.3620
Epoch 2/2
2246/2246 [==============================] - 5s 2ms/sample - loss: 2.4318 - acc: 0.3620
Test score: 2.4317893116790485
Test accuracy: 0.36197686


## Sequence Data Question
#### *Describe the `pad_sequences` method used on the training dataset. What does it do? Why do you need it?*

This method truncates and pads sequences to a specified length by removing values or adding dummy values till the length is reached.

## RNNs versus LSTMs
#### *What are the primary motivations behind using Long-ShortTerm Memory Cell unit over traditional Recurrent Neural Networks?*

LSTMs are able store information and make predictions on time-dependent data. Traditional RNNs are unable to effectively predict time-dependent data because the information is not saved to refer to later.

## RNN / LSTM Use Cases
#### *Name and Describe 3 Use Cases of LSTMs or RNNs and why they are suited to that use case*

Sentiment analysis - you can analyze the input of users to see trends in sentiment towards a product or service. An LSTM would be best suited for this because we want to see the feeling over time.

Text generation - you can predict what a user might input. Standard RNN would be sufficient because its dependent on position, but not time

Translation - you can translate between languages. A standard RNN would be suitable here for the same reason.



<a id="p2"></a>
## Part 2- CNNs

### Find the Frog

Time to play "find the frog!" Use Keras and ResNet50 (pre-trained) to detect which of the following images contain frogs:

<img align="left" src="https://d3i6fh83elv35t.cloudfront.net/newshour/app/uploads/2017/03/GettyImages-654745934-1024x687.jpg" width=400>


In [127]:
!wget https://cdn.pixabay.com/photo/2017/07/21/15/07/bull-frog-2525994__340.jpg
!wget https://cdn.pixabay.com/photo/2017/04/07/21/49/frog-2211972__340.jpg
!wget https://cdn.pixabay.com/photo/2017/07/14/17/44/frog-2504507_960_720.jpg
!wget https://cdn.pixabay.com/photo/2018/04/11/23/05/frog-3312038_960_720.jpg
!wget https://cdn.pixabay.com/photo/2019/07/10/16/56/frog-4329013_960_720.jpg

--2020-02-24 04:47:46--  https://cdn.pixabay.com/photo/2017/07/21/15/07/bull-frog-2525994__340.jpg
Resolving cdn.pixabay.com (cdn.pixabay.com)... 104.18.141.87, 104.18.82.97, 2606:4700::6812:5261, ...
Connecting to cdn.pixabay.com (cdn.pixabay.com)|104.18.141.87|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33412 (33K) [image/jpeg]
Saving to: ‘bull-frog-2525994__340.jpg.1’

bull-frog-2525994__ 100%[===================>]  32.63K  --.-KB/s    in 0.01s   

2020-02-24 04:47:46 (2.83 MB/s) - ‘bull-frog-2525994__340.jpg.1’ saved [33412/33412]

--2020-02-24 04:47:46--  https://cdn.pixabay.com/photo/2017/04/07/21/49/frog-2211972__340.jpg
Resolving cdn.pixabay.com (cdn.pixabay.com)... 104.18.82.97, 104.18.141.87, 2606:4700::6812:8d57, ...
Connecting to cdn.pixabay.com (cdn.pixabay.com)|104.18.82.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37944 (37K) [image/jpeg]
Saving to: ‘frog-2211972__340.jpg’

frog-2211972__340.j 100%[===========

In [128]:
# from google_images_download import google_images_download

# response = google_images_download.googleimagesdownload()
# arguments = {"keywords": "lilly frog pond", "limit": 99, "print_urls": True}
# absolute_image_paths = response.download(arguments)

At time of writing at least a few do, but since the Internet changes - it is possible your 5 won't. You can easily verify yourself, and (once you have working code) increase the number of images you pull to be more sure of getting a frog. Your goal is to validly run ResNet50 on the input images - don't worry about tuning or improving the model.

*Hint* - ResNet 50 doesn't just return "frog". The three labels it has for frogs are: `bullfrog, tree frog, tailed frog`

*Stretch goals* 
- Check for fish or other labels
- Create a matplotlib visualizations of the images and your prediction as the visualization label

In [2]:
# You've got something to do in this cell. ;)

import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

def process_img_path(img_path):
  return image.load_img(img_path, target_size=(224, 224))

def img_contains_frog(img):
    """ Scans image for Frogs
    
    Should return a boolean (True/False) if a frog is in the image.
    
    Inputs:
    ---------
    img:  Precrossed image ready for prediction. The `process_img_path`             function should already be applied to the image. 
    
    Returns: 
    ---------
    frogs (boolean):  TRUE or FALSE - There are frogs in the image.
    
    """
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    model = ResNet50(weights='imagenet')
    features = model.predict(x)
    results = decode_predictions(features, top=3)[0]
    print(results)
    for entry in results:
        if entry[1] == 'frog':
            return entry[2]
    return 0.0

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.i

In [14]:
import re, os

frog_dir = './Sprint/'

frogs = [process_img_path(frog_dir+f) for f in os.listdir(frog_dir) if re.search('.jpg', f)]


In [15]:
[img_contains_frog(frogs[ii]) for ii in range(len(frogs))]

40960/35363 [==================================] - 0s 1us/step
[('n01641577', 'bullfrog', 0.7258529), ('n01644900', 'tailed_frog', 0.14935401), ('n01644373', 'tree_frog', 0.1180797)]
[('n01641577', 'bullfrog', 0.96871424), ('n01644900', 'tailed_frog', 0.022436924), ('n01644373', 'tree_frog', 0.0033040792)]
[('n01641577', 'bullfrog', 0.26031822), ('n01737021', 'water_snake', 0.22164862), ('n04275548', 'spider_web', 0.13343656)]
[('n01641577', 'bullfrog', 0.6887148), ('n01644900', 'tailed_frog', 0.17964022), ('n01644373', 'tree_frog', 0.12558107)]
[('n01641577', 'bullfrog', 0.35916823), ('n01644900', 'tailed_frog', 0.31632495), ('n01737021', 'water_snake', 0.16264282)]


[0.0, 0.0, 0.0, 0.0, 0.0]

#### Stretch Goal: Displaying Predictions

In [ ]:
import matplotlib.pyplot as plt



<a id="p3"></a>
## Part 3 - Autoencoders

Describe a use case for an autoencoder given that an autoencoder tries to predict its own input. 

__*Your Answer:*__ 
This autoencoder might be used to predict what will be said/typed next to generate hints or useful information to help the user with what they are interested in.

<a id="p4"></a>
## Part 4 - More...

Answer the following questions, with a target audience of a fellow Data Scientist:

- What do you consider your strongest area, as a Data Scientist?
Troubleshooting; give me a problem and some time and I will get you an answer.

- What area of Data Science would you most like to learn more about, and why?
I would like to take some courses on the math of data analysis

- Where do you think Data Science will be in 5 years?
Highly automated but more demand for someone who understands the output or results of models


- What are the threats posed by AI to our society?
It has the potential to magnify wrongs or social inequalities. If we blindly rely on the technology, it has the potential to be used as a tool for supression and power consolidation. 

- How do you think we can counteract those threats?
Be diligent scientists and realize when there may be biases in data. Somehow you must minimalize/mitigate the effects of said biases.

- Do you think achieving General Artifical Intelligence is ever possible?
In the future, maybe, but its still 40+ years away, so it may be after our lifetime

A few sentences per answer is fine - only elaborate if time allows.

## Congratulations! 

Thank you for your hard work, and congratulations! You've learned a lot, and you should proudly call yourself a Data Scientist.


In [16]:
from IPython.display import HTML

HTML("""<iframe src="https://giphy.com/embed/26xivLqkv86uJzqWk" width="480" height="270" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/mumm-champagne-saber-26xivLqkv86uJzqWk">via GIPHY</a></p>""")